# Spikes in microelectrode arrays (MEAs)

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import LFPy
import neuron
from neuron import h
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell

np.random.seed(12345)
ns.load_mechs_from_folder(ns.cell_models_folder)

tstop = 150
dt = 2**-5

# Time window to extract spike from:
t0 = 118
t1 = 128

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

In [ ]:
cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
ns.point_axon_down(cell)

cell.set_rotation(y=np.pi/2)
cell.set_rotation(x=-np.pi/2)
cell.set_rotation(y=-0.02)

cell.set_pos(z=80)

syn, cell = insert_current_stimuli(cell)
cell.simulate(rec_imem=True, rec_vmem=True)
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]


In [ ]:
# Create a grid of measurement locations, in (mum)
X = np.arange(0, 1001, 500)
Y = np.zeros(X.shape)
Z = np.zeros(X.shape) + 0

# Define electrode parameters
MEA_elec_params = {
    'sigma_T': 0.3,      # extracellular conductivity
    'sigma_G': 0.0,      # MEA glass electrode plate conductivity
    'sigma_S': 1.5,      # Saline bath conductivity
    'x': X.flatten(),  # electrode requires 1d vector of positions
    'y': Y.flatten(),
    'z': Z.flatten(),
    "method": "pointsource",
}
# Define electrode parameters
semiinf_elec_params = {
    'sigma_T': 0.3,      # extracellular conductivity
    'sigma_G': 0.0,      # MEA glass electrode plate conductivity
    'sigma_S': 0.3,      # Saline bath conductivity
    'x': X.flatten(),  # electrode requires 1d vector of positions
    'y': Y.flatten(),
    'z': Z.flatten(),
    "method": "pointsource",
}
# Define electrode parameters
homog_elec_params = {
    'sigma_T': 0.3,      # extracellular conductivity
    'sigma_G': 0.3,      # MEA glass electrode plate conductivity
    'sigma_S': 0.3,      # Saline bath conductivity
    'x': X.flatten(),  # electrode requires 1d vector of positions
    'y': Y.flatten(),
    'z': Z.flatten(),
    "method": "pointsource",
}


In [ ]:
MEA = LFPy.RecMEAElectrode(cell, **MEA_elec_params)
semi_inf = LFPy.RecMEAElectrode(cell, **semiinf_elec_params)
homog = LFPy.RecMEAElectrode(cell, **homog_elec_params)

MEA.data = MEA.get_transformation_matrix() @ cell.imem
semi_inf.data = semi_inf.get_transformation_matrix() @ cell.imem
homog.data = homog.get_transformation_matrix() @ cell.imem

In [ ]:
def plot_recording_set_up(cell, ax_neur, ax_side, MEA,
                          syn_idx, cell_plot_colors):
            
    for idx in range(len(MEA.x)):
        ax_side.plot(MEA.x[idx], MEA.z[idx] - 10, 's', clip_on=False,
                     c='k', zorder=10, mec='none', ms=5)
        ax_side.plot([MEA.x[idx], MEA.x[idx]], [MEA.z[idx] - 10, MEA.z[idx] - 150],
                     c='k', zorder=10, lw=2)
        ax_neur.plot(MEA.x[idx], MEA.y[idx], 's', c='k', zorder=10, ms=5)
        ax_side.text(MEA.x[idx], -75, "x=%d µm" % MEA.x[idx], ha="center", va="top")

    ax_side.axhspan(-250, 0, facecolor='0.5', edgecolor='none')
    ax_side.axhspan(0, MEA.h, facecolor='#FFB380', edgecolor='none')
    ax_side.axhspan(MEA.h, MEA.h + 250, facecolor='#AFE9ED', edgecolor='none')
    ax_neur.axhspan(-500, 500, facecolor='#FFB380', edgecolor='none')

    ax_neur.arrow(-220, +100, 30, 0, lw=1, head_width=12, color='k', clip_on=False)
    ax_neur.arrow(-220, +100, 0, 30, lw=1, head_width=12, color='k', clip_on=False)
    ax_neur.text(-140, +100, 'x', ha='center', va='center', clip_on=False)
    ax_neur.text(-220, +200, 'y', ha='center', va='center', clip_on=False)

    ax_side.arrow(-220, 180, 30, 0, lw=1, head_width=12, color='k', clip_on=False)
    ax_side.text(-140, 185, 'x', ha='center', va='center')
    ax_side.arrow(-220, 180, 0, 30, lw=1, head_width=12, color='k', clip_on=False)
    ax_side.text(-220, 265, 'z', ha='center', va='center')

    ax_side.plot([-500, 1250], [MEA.h, MEA.h], color='k')
    ax_side.plot([-500, 1250], [0, 0], 'k')  # PLOTTING BOTTOM OF MEA

    ax_side.plot([1280, 1280], [0, MEA.h], '_-',
                 color='k', lw=1, clip_on=False, solid_capstyle='butt')
    ax_side.text(1300, MEA.h / 2, '%g\nµm' % MEA.h, va='center')

    ax_side.text(1200, -10, 'MEA', va='top', ha='right')
    ax_side.text(1200, MEA.h, 'aCSF', va='bottom', ha='right')
    ax_side.text(1200, MEA.h - 25, 'tissue', va='top', ha='right')

    ax_side.axis([-300, 1250, -70, MEA.h + 70])
    ax_neur.axis([-300, 1250, -250, 250])
    for comp in range(cell.totnsegs):
        if comp == 0:
            ax_neur.scatter(cell.x[comp].mean(), cell.y[comp].mean(), 
                            s=cell.d[comp], edgecolor='none', color='gray', zorder=1)
            ax_side.scatter(cell.x[comp].mean(), cell.z[comp].mean(), 
                            s=cell.d[comp], edgecolor='none', color='gray', zorder=1)
        else:
            ax_neur.plot(cell.x[comp], cell.y[comp],
                         lw=cell.d[comp] / 2, color='gray', zorder=1)
            ax_side.plot(cell.x[comp], cell.z[comp],
                         lw=cell.d[comp] / 2, color='gray', zorder=1)

syn_idx = syn.idx

cell_plot_idxs = [syn_idx]
cell_plot_colors = {syn_idx: 'y'}
num_cols = 4

fig = plt.figure(figsize=[6, 2.2])
plt.subplots_adjust(hspace=0.6, wspace=0.4, right=0.99, left=0.03, top=0.87, bottom=0.22)
ax1 = fig.add_axes([0.02, 0.0, 0.4, 0.47], aspect=1, frameon=False,
                   xticks=[], yticks=[], title='top view')
ax3 = fig.add_axes([0.02, 0.5, 0.4, 0.47], aspect=1, frameon=False,
                   xticks=[], yticks=[], title='side view')

ax_ec1 = fig.add_subplot(1, 6, 4, 
                        xlabel='time (ms)', ylabel='µV',
                        title='x=%d µm'% MEA.x[0])
ax_ec2 = fig.add_subplot(1, 6, 5, 
                        xlabel='time (ms)', ylabel=r'µV',
                        title='x=%d µm'% MEA.x[1])
ax_ec3 = fig.add_subplot(1, 6, 6,
                        xlabel='time (ms)', ylabel='µV',
                        title='x=%d µm'% MEA.x[2])

l_elec = plot_recording_set_up(cell, ax1, ax3, MEA,
                                      syn_idx, cell_plot_colors)

for elec in range(len(MEA.x)):
    ax = [ax_ec1, ax_ec2, ax_ec3][elec]
    l3, = ax.plot(cell.tvec, 1000 * (homog.data[elec]), lw=1.5, c='gray', ls="-")
    l1, = ax.plot(cell.tvec, 1000 * (MEA.data[elec]), lw=1.5, c='k')
    l2, = ax.plot(cell.tvec, 1000 * (semi_inf.data[elec]), lw=1.5, c='r', ls="--")

fig.legend([l1, l2, l3],
           ["slice in aCSF", "semi-infinite", "infinite"],
           frameon=False, numpoints=1, ncol=4, loc=(0.47, 0.0))
simplify_axes([ax_ec1, ax_ec2, ax_ec3])
mark_subplots([ax3, ax1, ax_ec1, ax_ec2, ax_ec3], ypos=1.15, xpos=-0.01)

plt.savefig('example_MEA.pdf')